In [37]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import pipeline
import torch
import gradio as gr

## **Load GPT-2 for Generation**

In [38]:
print("🔁 Loading GPT-2 model...")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model.config.pad_token_id = gpt2_tokenizer.eos_token_id

def generate_fake_news(prompt="Breaking News:"):
    if not prompt.strip():
        return "⚠️ No topic provided for generation."

    input_ids = gpt2_tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = torch.ones_like(input_ids)

    outputs = gpt2_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=30,
        num_return_sequences=5,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )

    headlines = []
    for output in outputs:
        headline = gpt2_tokenizer.decode(output, skip_special_tokens=True)
        headlines.append(headline)
    return "\n".join([f"{i+1}. {h}" for i, h in enumerate(headlines)])

🔁 Loading GPT-2 model...


## **Load BERT for Detection**

In [39]:
print("🔁 Loading BERT classifier pipeline...")
clf = pipeline(
    "text-classification",
    model="jy46604790/Fake-News-Bert-Detect",
    tokenizer="jy46604790/Fake-News-Bert-Detect"
)

def detect_fake_news(text):
    if not text.strip():
        return "⚠️ No news text provided for detection."

    result = clf(text[:500])[0]  # truncate long texts
    label = "🟥 FAKE" if result['label'] == "LABEL_0" else "🟩 REAL"
    confidence = round(result['score'] * 100, 2)
    return f"📝 Input: {text}\n\n🔍 Prediction: {label}\n✅ Confidence: {confidence}%"

🔁 Loading BERT classifier pipeline...


Device set to use cpu


## **Gradio Interface Function**

In [40]:
def interface(prompt, text):
    fake_news = generate_fake_news(prompt) if prompt.strip() else "⚠️ No topic entered."
    detection = detect_fake_news(text) if text.strip() else "⚠️ No news text entered."
    return fake_news, detection

## **Gradio App**

In [44]:
demo = gr.Interface(
    fn=interface,
    inputs=[
        gr.Textbox(label="🧠 Enter a Topic for Fake News", placeholder="e.g. Budget 2025, Alien Invasion"),
        gr.Textbox(label="📄 Paste News to Detect", placeholder="e.g. Government announces free petrol every Monday...")
    ],
    outputs=[
        gr.Textbox(label="📰 Generated Fake News"),
        gr.Textbox(label="🕵️‍♂️ Detection Result")
    ],
    title=" Fake News Generator & Detector using Generative AI and NLP",
    description="\n",
    allow_flagging="never"
)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [ ]:
demo.launch()

![Image of output](image1.png)

![Image of Output](image2.png)